###  Import Libraries

In [236]:
import numpy as np
import pandas as pd 
import datetime
from pivottablejs import pivot_ui
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns 
import pandas_profiling
import missingno as msno 

### Load Dataset

In [321]:
smart_home = pd.read_csv(r"../Smart Home Dataset with weather Information/HomeC.csv")

C:\Users\Cornel\AppData\Local\Temp\ipykernel_14460\2174768821.py:1: DtypeWarning: Columns (0,27) have mixed types. Specify dtype option on import or set low_memory=False.
  smart_home = pd.read_csv(r"../Smart Home Dataset with weather Information/HomeC.csv")


In [322]:
#check the number of rows and columns
def print_dim(df):
    '''
    Function to print the dimensions of a given python dataframe
    Required Input -
        - df = Pandas DataFrame
    Expected Output -
        - Data size
    '''
    print("Data size: Rows-{0} Columns-{1}".format(df.shape[0],df.shape[1]))
print_dim(smart_home)

Data size: Rows-503911 Columns-32


### Basic Preprocessing 

In [323]:
#to find datatype that is not object and/or float.
result = [col for col in smart_home.columns if smart_home[col].dtype != "object" and smart_home[col].dtype != "float"]
if result == []:
    print ("None: all datatypes are either float or object")
else:
    print(result)

None: all datatypes are either float or object


In [324]:
#put the datatypes in perspective
def view_datatypes_in_perspective(df):
    '''
    Function to group dataframe columns into three common dtypes and visualize the columns
    Required Input - 
        - df = Pandas DataFrame
    Expected Output -
        - three unique datatypes (float, object, others(for the rest))
    '''
    float = 0
    float_col = []
    object = 0
    object_col = []
    others = 0
    others_col = []
    for col in df.columns:
        if df[col].dtype ==  "float":
            float += 1
            float_col.append(col) 
        elif df[col].dtypes == "object":
            object += 1
            object_col.append(col)
        else:
            others +=1
            others_col.append(col)
            others_col.append(smart_home[col].dtype)        
    print (f" float = {float} \t{float_col}, \n \nobject = {object} \t{object_col}, \n\nothers = {others} \t{others_col} ")

view_datatypes_in_perspective(smart_home)

 float = 28 	['use [kW]', 'gen [kW]', 'House overall [kW]', 'Dishwasher [kW]', 'Furnace 1 [kW]', 'Furnace 2 [kW]', 'Home office [kW]', 'Fridge [kW]', 'Wine cellar [kW]', 'Garage door [kW]', 'Kitchen 12 [kW]', 'Kitchen 14 [kW]', 'Kitchen 38 [kW]', 'Barn [kW]', 'Well [kW]', 'Microwave [kW]', 'Living room [kW]', 'Solar [kW]', 'temperature', 'humidity', 'visibility', 'apparentTemperature', 'pressure', 'windSpeed', 'windBearing', 'precipIntensity', 'dewPoint', 'precipProbability'], 
 
object = 4 	['time', 'icon', 'summary', 'cloudCover'], 

others = 0 	[] 


In [325]:
#visualize the non-float datatypes to check for anomaly
smart_home[["time", "icon", "summary", "cloudCover"]].head(10)
##NB: cloudCover has both numeric and non-numeric values. 
##NB: the [:57] values in cloudCover are labelled "cloudCover" instead of float types. They would be replaced with NaN

,time,icon,summary,cloudCover
0,1451624400,clear-night,Clear,cloudCover
1,1451624401,clear-night,Clear,cloudCover
2,1451624402,clear-night,Clear,cloudCover
3,1451624403,clear-night,Clear,cloudCover
4,1451624404,clear-night,Clear,cloudCover
5,1451624405,clear-night,Clear,cloudCover
6,1451624406,clear-night,Clear,cloudCover
7,1451624407,clear-night,Clear,cloudCover
8,1451624408,clear-night,Clear,cloudCover
9,1451624409,clear-night,Clear,cloudCover


In [225]:
def do_data_profiling(df, filename):
    '''
    Function to do basic data profiling
    Required Input - 
        - df = Pandas DataFrame
        - filename = Path for output file with a .html extension
    Expected Output -
        - HTML file with data profiling summary
    '''
    profile = pandas_profiling.ProfileReport(df)
    profile.to_file(output_file = filename)
    print("Data profiling done")
    
do_data_profiling(smart_home, "smart_home_profile.csv")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

c:\Users\Cornel\anaconda3\lib\site-packages\pandas_profiling\profile_report.py:314: UserWarning: Extension .csv not supported. For now we assume .html was intended. To remove this warning, please use .html or .json.
  warnings.warn(


Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Data profiling done


In [241]:
'''
            RESULT FROM THE DATA PROFILING
1. There are NaN values present (at least) in the last row
2. cloudCover has both numeric and non-numeric values. The non-numeric values will be converter to NaN, and fillna
3. 

'''

'\nresult from\n\n'

In [326]:
#pre-process all columns
#drop Nan values
#convert to numeric

#function to replace non-numeric values in the all column
def replace_non_numeric(df: pd.DataFrame, columns):
    """
    Replaces non-numeric values in the specified columns of a Pandas dataframe with NaN.

    Parameters:
        df (pd.DataFrame): The dataframe to process.
        columns (list): A list of column names to replace non-numeric values in.

    Returns:
        pd.DataFrame: The updated dataframe with non-numeric values replaced by NaN.
    """
    for col in columns:
        df.dropna(subset = col, inplace= True)
        if df[col].dtype == 'object' or df[col].dtype == 'float':
            df[col] = pd.to_numeric(df[col], errors='coerce')
            # df.dropna(subset = col, inplace= True)
        else:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            # df.dropna(subset = col, inplace= True)
    return df

smart_home = replace_non_numeric(smart_home, columns= ['time', 'use [kW]', 'gen [kW]', 'House overall [kW]', 'Dishwasher [kW]',
       'Furnace 1 [kW]', 'Furnace 2 [kW]', 'Home office [kW]', 'Fridge [kW]',
       'Wine cellar [kW]', 'Garage door [kW]', 'Kitchen 12 [kW]',
       'Kitchen 14 [kW]', 'Kitchen 38 [kW]', 'Barn [kW]', 'Well [kW]',
       'Microwave [kW]', 'Living room [kW]', 'Solar [kW]', 'temperature',
       'humidity', 'visibility', 'apparentTemperature',
       'pressure', 'windSpeed', 'cloudCover', 'windBearing', 'precipIntensity',
       'dewPoint', 'precipProbability'])

smart_home.shape 

(503910, 32)

In [327]:
#figuring out how to fill the NaN values in cloudCover. It is important to note that cloudCover has a strong relationship
    #with apparentTemperature.
smart_home[(smart_home["apparentTemperature"] == 29.26) & (smart_home["cloudCover"].notnull())][:10]

#from the code above, apparentTemperature at 29.26 (the same value when cloudCover was NaN) also had non-Nan values of 
    #cloudCover set at 0.1. This value will used to replace cloudCover NaN values below. 

,time,use [kW],gen [kW],House overall [kW],Dishwasher [kW],Furnace 1 [kW],Furnace 2 [kW],Home office [kW],Fridge [kW],Wine cellar [kW],...,visibility,summary,apparentTemperature,pressure,windSpeed,cloudCover,windBearing,precipIntensity,dewPoint,precipProbability
467603,1.452092e+09,0.775083,0.004750,0.775083,0.000000,0.019933,0.083617,0.042283,0.017950,0.007800,...,10.0,Clear,29.26,1023.27,9.86,0.1,331.0,0.0,20.03,0.0
467604,1.452092e+09,0.871583,0.004783,0.871583,0.000017,0.020367,0.166967,0.042233,0.005233,0.007950,...,10.0,Clear,29.26,1023.27,9.86,0.1,331.0,0.0,20.03,0.0
467605,1.452092e+09,1.288233,0.004767,1.288233,0.000050,0.021200,0.511183,0.041917,0.005450,0.008133,...,10.0,Clear,29.26,1023.27,9.86,0.1,331.0,0.0,20.03,0.0
467606,1.452092e+09,1.418550,0.004767,1.418550,0.000083,0.021267,0.619317,0.041750,0.005500,0.008467,...,10.0,Clear,29.26,1023.27,9.86,0.1,331.0,0.0,20.03,0.0
467607,1.452092e+09,1.424717,0.004800,1.424717,0.000067,0.021583,0.617350,0.041817,0.005900,0.008267,...,10.0,Clear,29.26,1023.27,9.86,0.1,331.0,0.0,20.03,0.0
467608,1.452092e+09,1.418033,0.004767,1.418033,0.000067,0.021250,0.615900,0.041767,0.005467,0.008183,...,10.0,Clear,29.26,1023.27,9.86,0.1,331.0,0.0,20.03,0.0
467609,1.452092e+09,1.421767,0.004800,1.421767,0.000083,0.021533,0.614717,0.041850,0.005500,0.008383,...,10.0,Clear,29.26,1023.27,9.86,0.1,331.0,0.0,20.03,0.0
467610,1.452092e+09,1.404133,0.004783,1.404133,0.000067,0.021317,0.599750,0.041817,0.005483,0.008217,...,10.0,Clear,29.26,1023.27,9.86,0.1,331.0,0.0,20.03,0.0
467611,1.452092e+09,1.270833,0.004800,1.270833,0.000067,0.021467,0.555083,0.053867,0.005450,0.008450,...,10.0,Clear,29.26,1023.27,9.86,0.1,331.0,0.0,20.03,0.0
467612,1.452092e+09,0.934050,0.004833,0.934050,0.000017,0.021017,0.278467,0.114467,0.005633,0.008250,...,10.0,Clear,29.26,1023.27,9.86,0.1,331.0,0.0,20.03,0.0


In [338]:
#fill Nan Values in the cloudCover column
def treat_missing_numeric(df,columns,how = 'mean', value = None):
    '''
    Function to treat missing values in numeric columns
    Required Input - 
        - df = Pandas DataFrame
        - columns = List input of all the columns need to be imputed
        - how = valid values are 'mean', 'mode', 'median','ffill', numeric value
    Expected Output -
        - Pandas dataframe with imputed missing value in mentioned columns
    '''
    if how == 'mean':
        for i in columns:
            print("Filling missing values with mean for columns - {0}".format(i))
            df[i] = df[i].fillna(df[i].mean())
            
    elif how == 'mode':
        for i in columns:
            print("Filling missing values with mode for columns - {0}".format(i))
            df[i] = df[i].fillna(df[i].mode())
    
    elif how == 'median':
        for i in columns:
            print("Filling missing values with median for columns - {0}".format(i))
            df[i] = df[i].fillna(df[i].median())
    
    elif how == 'ffill':
        for i in columns:
            print("Filling missing values with forward fill for columns - {0}".format(i))
            df[i] = df[i].fillna(method ='ffill')
    
    elif how == 'digit':
        for i in columns:
            print("Filling missing values with {0} for columns - {1}".format(how, i))
            df[i] = df[i].fillna(str(value)) 
      
    else:
        print("Missing value fill cannot be completed")
    return df.head(5)

treat_missing_numeric(smart_home, ["cloudCover"], how="digit", value = 0.1)  

Filling missing values with digit for columns - cloudCover


,time,use [kW],gen [kW],House overall [kW],Dishwasher [kW],Furnace 1 [kW],Furnace 2 [kW],Home office [kW],Fridge [kW],Wine cellar [kW],...,visibility,summary,apparentTemperature,pressure,windSpeed,cloudCover,windBearing,precipIntensity,dewPoint,precipProbability
0,1.451624e+09,0.932833,0.003483,0.932833,0.000033,0.020700,0.061917,0.442633,0.124150,0.006983,...,10.0,Clear,29.26,1016.91,9.18,0.1,282.0,0.0,24.4,0.0
1,1.451624e+09,0.934333,0.003467,0.934333,0.000000,0.020717,0.063817,0.444067,0.124000,0.006983,...,10.0,Clear,29.26,1016.91,9.18,0.1,282.0,0.0,24.4,0.0
2,1.451624e+09,0.931817,0.003467,0.931817,0.000017,0.020700,0.062317,0.446067,0.123533,0.006983,...,10.0,Clear,29.26,1016.91,9.18,0.1,282.0,0.0,24.4,0.0
3,1.451624e+09,1.022050,0.003483,1.022050,0.000017,0.106900,0.068517,0.446583,0.123133,0.006983,...,10.0,Clear,29.26,1016.91,9.18,0.1,282.0,0.0,24.4,0.0
4,1.451624e+09,1.139400,0.003467,1.139400,0.000133,0.236933,0.063983,0.446533,0.122850,0.006850,...,10.0,Clear,29.26,1016.91,9.18,0.1,282.0,0.0,24.4,0.0


In [339]:
#convert timestamp to datetime format
def convert_timestamp(ts):
    """
    Converts a Unix timestamp to a formatted date and time string.

    Args:
        ts (int): The Unix timestamp to convert.

    Returns:
        str: A formatted date and time string in the format 'YYYY-MM-DD HH:MM:SS'.
    """
    utc_datetime = datetime.datetime.utcfromtimestamp(ts)
    formatted_datetime = utc_datetime.strftime('%Y-%m-%d %H:%M:%S')
    formatted_datetime = pd.to_datetime(formatted_datetime, infer_datetime_format=True) 
    return formatted_datetime

    
# Use apply() to convert the entire timestamp column to formatted strings
smart_home['time'] = smart_home['time'].apply(convert_timestamp)
smart_home["time"].head(4)

0   2016-01-01 05:00:00
1   2016-01-01 05:00:01
2   2016-01-01 05:00:02
3   2016-01-01 05:00:03
Name: time, dtype: datetime64[ns]

In [340]:
#Recheck the datatypes
float = 0
object = 0
object_col = []
others = 0
others_col = []
for col in smart_home.columns:
    if smart_home[col].dtype ==  "float":
        float += 1
    elif smart_home[col].dtypes == "object":
        object += 1
        object_col.append(col)
    else:
        others +=1
        others_col.append(col)
        others_col.append(smart_home[col].dtype)
        
print (f" float = {float}, object = {object} {object_col}, and others = {others} {others_col} ")

 float = 28, object = 3 ['icon', 'summary', 'cloudCover'], and others = 1 ['time', dtype('<M8[ns]')] 


In [350]:
#recheck for NaN values
def view_NaN(df):
    """
    Prints the name of any column in a Pandas DataFrame that contains NaN values.

    Parameters:
        - df: Pandas DataFrame

    Returns:
        - None
    """
    for col in df.columns:
        if df[col].isnull().any() == True:
            print("there is NaN present in column:", col)
        else:
            print("No NaN present in column:", col)


view_NaN(smart_home)

No NaN present time
No NaN present use [kW]
No NaN present gen [kW]
No NaN present House overall [kW]
No NaN present Dishwasher [kW]
No NaN present Furnace 1 [kW]
No NaN present Furnace 2 [kW]
No NaN present Home office [kW]
No NaN present Fridge [kW]
No NaN present Wine cellar [kW]
No NaN present Garage door [kW]
No NaN present Kitchen 12 [kW]
No NaN present Kitchen 14 [kW]
No NaN present Kitchen 38 [kW]
No NaN present Barn [kW]
No NaN present Well [kW]
No NaN present Microwave [kW]
No NaN present Living room [kW]
No NaN present Solar [kW]
No NaN present temperature
No NaN present icon
No NaN present humidity
No NaN present visibility
No NaN present summary
No NaN present apparentTemperature
No NaN present pressure
No NaN present windSpeed
No NaN present cloudCover
No NaN present windBearing
No NaN present precipIntensity
No NaN present dewPoint
No NaN present precipProbability


### Exploratory Data Analysis (EDA)

### Time Series Analysis